In [ ]:
# some reference material
# 1) https://www.r-bloggers.com/2015/09/selecting-the-number-of-neurons-in-the-hidden-layer-of-a-neural-network/
#    #:~:text=The%20most%20common%20rule%20of,number%20is%20greater%20than%201).
# 2) https://www.analyticsvidhya.com/blog/2021/08/
#    a-walk-through-of-regression-analysis-using-artificial-neural-networks-in-tensorflow/
# 

In [104]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import feature_selection
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import r2_score
from sklearn.impute import KNNImputer

def impute(df_train, df_test, method):
    # There are  missing values in our data set lets replace them with the mean
    
    #df_train = df_train.fillna(df_train.mean())
    #df_test = df_test.fillna(df_test.mean())
    
    if method == 'KNN':
        imputer_train = KNNImputer(n_neighbors = 2)
        df_train = imputer_train.fit_transform(df_train.copy())

        imputer_test = KNNImputer(n_neighbors = 2)
        df_test = imputer_test.fit_transform(df_test.copy())
    
    return pd.DataFrame(df_train), pd.DataFrame(df_test)
    
    #Outlier detection
def filter(df1, df2):
  # Filter feature selection
  from sklearn.feature_selection import VarianceThreshold 
  
  # Step 1: Removing Constant features
  constant_filter = VarianceThreshold(threshold=0)
  data_constant = constant_filter.fit_transform(df1)
  #print(data_constant.shape)
  constant_columns = [column for column in df1.columns if column not in df1.columns[constant_filter.get_support()]]
  data_cons1 = df1.drop(constant_columns,axis=1)
  data_cons2 = df2.drop(constant_columns,axis=1)
      
  # Step 2: Removing Quasi-Constant Features
  qcons_filter = VarianceThreshold(threshold=0.001)
  data_qcons = qcons_filter.fit_transform(df1)
  #print(data_qcons.shape)
  qcons_columns = [column for column in df1.columns if column not in df1.columns[qcons_filter.get_support()]]
  data_qcons1 = df1.drop(qcons_columns,axis=1)
  data_qcons2 = df2.drop(qcons_columns,axis=1)
  data_qcons_t1 = data_qcons1.T
  data_qcons_t2 = data_qcons2.T 
  
  # Step 3: Removing Duplicate Columns
  data_cons_dup1 = data_qcons_t1.drop_duplicates(keep='first').T
  data_cons_dup2 = data_qcons_t2.drop_duplicates(keep='first').T

  return data_cons_dup1, data_cons_dup2

def reduce_feats(X):
    ### performs PCA on the preprocessed features ###
    pca = PCA(n_components=0.25, whiten=True)
    return pca.fit_transform(X)

def test_models(X_train_model, y_train_model, X_val, y_val):
    bayes_ridge = BayesianRidge()
    bayes_ridge2 = BayesianRidge(alpha_init = 1e-2, lambda_init = 1e-3)
    linear = LinearRegression()
    
    models = [bayes_ridge, bayes_ridge2, linear]
    scores = []
    for model in models:
        model.fit(X_train_model, y_train_model)
        y_pred = model.predict(X_val)
        scores.append(r2_score(y_val, y_pred))
        
    print(scores)    
    return models, scores



In [108]:
X_test = pd.read_csv("X_test.csv", index_col=0).values
X_train = pd.read_csv("X_train.csv", index_col=0).values
y_train = pd.read_csv("y_train.csv", index_col=0).values

df_train_orig = pd.DataFrame(X_train)
df_test_orig = pd.DataFrame(X_test)

# There are  missing values in our data set, let's carry out data imputation
df_train, df_test = impute(df_train_orig, df_test_orig, 'KNN')
print('Check if any nan left:', df_train.isnull().values.any(), df_test.isnull().values.any())

# Apply std scaler and reduce dimensions 
X_train, X_test = filter(df_train, df_test)
X_train, X_test = X_train.values, X_test.values

X_scaler = StandardScaler()
y_scaler = StandardScaler()

X_train_scaled = X_scaler.fit_transform(X_train)
X_train = reduce_feats(X_train_scaled )
#y_train_scaled = y_scaler.fit_transform(y_train)

print(X_train.shape)

# Split the data into training set and testing set
X_train_model, X_val, y_train_model, y_val = train_test_split(X_train, y_train, 
                                                              test_size = 0.25, random_state = 42)
y_train_model = np.ravel(y_train_model)
y_val = np.ravel(y_val)

# Test the models on the validation set we have from splitting the data
test_models(X_train_model, y_train_model, X_val, y_val)

# Based on the testing commented above, pick a model for the final prediction

Check if any nan left: False False
(1212, 23)
[0.3393211103243102, 0.3393211098516762, 0.3375158904859149]


([BayesianRidge(),
  BayesianRidge(alpha_init=0.01, lambda_init=0.001),
  LinearRegression()],
 [0.3393211103243102, 0.3393211098516762, 0.3375158904859149])

In [ ]:
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.losses import MeanSquaredLogarithmicError

def build_nn():
    #unit array containing the units for the first, second and third hidden layer
    units = np.array([512, 256, 256])
    nn_model = Sequential([Dense(units[0], kernel_initializer = 'random_normal', activation = 'relu'), Dropout(0.25) 
                        Dense(units[1], kernel_initializer = 'random_normal', activation = 'relu'), Dropout(0.25)
                        Dense(units[2], kernel_initializer = 'random_normal', activation = 'relu'), 
                        Dense(1, acivation = 'linear')])
    return nn_model

regressor_nn = build_nn()
regressor_nn.compile(loss = msle, optimizer=Adam(learning_rate=learning_rate), metrics=[msle])

In [95]:
"""TESTING KNN




    TEEEESTIIINGGG """

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import feature_selection
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import r2_score
from sklearn.impute import KNNImputer

X_test = pd.read_csv("X_test.csv", index_col=0).values
X_train = pd.read_csv("X_train.csv", index_col=0).values
y_train = pd.read_csv("y_train.csv", index_col=0).values

df_train_orig = pd.DataFrame(X_train)
df_test_orig = pd.DataFrame(X_test)


def impute2(df_train, df_test, method, neighbors):
    # There are  missing values in our data set lets replace them with the mean
    if method == 'mean':
        df_train = df_train.fillna(df_train.copy().mean())
        df_test = df_test.fillna(df_test.copy().mean())
    
    elif method == 'KNN':
        imputer_train = KNNImputer(n_neighbors = neighbors)
        df_train = imputer_train.fit_transform(df_train.copy())

        imputer_test = KNNImputer(n_neighbors = neighbors)
        df_test = imputer_test.fit_transform(df_test.copy())
        
    print('Check original unmodified', df_train_orig.isnull().values.any(), df_test_orig.isnull().values.any())
    
    return pd.DataFrame(df_train), pd.DataFrame(df_test)


neighbors = range(2, 50)
scores = []
for n in neighbors:

    df_train, df_test = impute2(df_train_orig, df_test_orig, 'KNN', n)
    # Apply std scaler and reduce dimensions 
    X_train, X_test = filter(df_train, df_test)
    X_train, X_test = X_train.values, X_test.values

    X_scaler = StandardScaler()
    y_scaler = StandardScaler()

    X_train_scaled = X_scaler.fit_transform(X_train)
    X_train = reduce_feats(X_train_scaled )
    #y_train_scaled = y_scaler.fit_transform(y_train)

    print(X_train.shape)

    # Split the data into training set and testing set
    X_train_model, X_val, y_train_model, y_val = train_test_split(X_train, y_train, 
                                                                  test_size = 0.2, random_state = 42)
    y_train_model = np.ravel(y_train_model)
    y_val = np.ravel(y_val)
    
    print(n)
    mods, rvals = test_models(X_train_model, y_train_model, X_val, y_val)
    scores.append(rvals[0])
    
plt.plot(neighbors, np.array(scores))
plt.show()

### Note: 2 neighbors best for KNN

Check original unmodified True True
(1212, 23)
2
[0.3170788661812989, 0.31707886505090377, 0.3130501520901099]


KeyboardInterrupt: 

In [96]:
df_train, df_test = impute2(df_train_orig, df_test_orig, 'mean', 1)
print('Check new no nan', df_train.isnull().values.any(), df_test.isnull().values.any())
X_train, X_test = filter(df_train, df_test)
X_train, X_test = X_train.values, X_test.values

X_scaler = StandardScaler()
y_scaler = StandardScaler()

X_train_scaled = X_scaler.fit_transform(X_train)
X_train = reduce_feats(X_train_scaled )
#y_train_scaled = y_scaler.fit_transform(y_train)

print(X_train.shape)

# Split the data into training set and testing set
X_train_model, X_val, y_train_model, y_val = train_test_split(X_train, y_train, 
                                                              test_size = 0.2, random_state = 42)
y_train_model = np.ravel(y_train_model)
y_val = np.ravel(y_val)

mods, rvals = test_models(X_train_model, y_train_model, X_val, y_val)

Check original unmodified True True
Check new no nan False False
(1212, 22)
[0.31377220601580125, 0.31377220523913363, 0.3101474492547174]


In [59]:
print(np.var(y_val))
print(1/np.var(y_val))

89.97805212620027
0.011113821386102389


In [29]:
print(X_train.shape)

(1212, 773)


In [ ]:
from sklearn.metrics import r2_score
score = r2_score(y, y_pred)